<a href="https://colab.research.google.com/github/MaximeCornaton/CPE_IA-DeepLearning/blob/master/DEPOs_TP1_TP2/TP2/TP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scikit-learn matplotlib

# TP2

## Exercice 2.1

In [ ]:
import numpy as np

class Perceptron:

    def __init__(self, num_inputs, activation_function, learning_rate, num_epochs):
        self.__num_inputs = num_inputs
        # Créer un tableau de longueur num_inputs contenant les poids (entre 0 et 1) associés aux entrées
        self.__weights = np.random.rand(num_inputs)
        # Choix d'un biais random (entre 0 et 1)
        self.__bias = np.random.rand()
        self.__activation_function = activation_function
        self.__learning_rate = learning_rate
        self.__num_epochs = num_epochs

    def train(self, X, y):
        for epoch in range(self.__num_epochs):
            for i in range(len(X)):
                input_data = X[i]
                prediction = self.predict(input_data)
                error = y[i] - prediction
                self.__weights += self.__learning_rate * error * input_data
                self.__bias += self.__learning_rate * error
        
    def predict(self, X):
        linear_combination = self.__bias
        for i in range(len(X)):
            linear_combination += X[i] * self.__weights[i]
        return self.activate(linear_combination)

    def activate(self, x):
        if self.__activation_function == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.__activation_function == 'tanh':
            return np.tanh(x)
        elif self.__activation_function == 'relu':
            return np.maximum(0, x)
        elif self.__activation_function == 'gaussian':
            return np.exp(-x ** 2)
        else:
            # Renvoyer la fonction d'activation identité par défaut
            return x




if __name__ == "__main__":
    # Exemple de test avec une fonction SIGMOID
    # L'objectif est de démontrer la capacité du perceptron à apprendre la logique OR
    X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    y = np.array([0, 1, 1, 1])

    perceptron = Perceptron(num_inputs=2, activation_function='sigmoid', learning_rate=0.5, num_epochs=1000)
    perceptron.train(X, y)

    # Test des prédictions
    test_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    for data in test_data:
        prediction = perceptron.predict(data)
        print(f"Prévision pour {data}: {prediction}")

## Exercice 2.2

## Exercice 2.3

## Exercice 2.4